In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

In [19]:
import pandas as pd
import numpy as np

In [20]:
data = pd.read_csv("Training.csv")

In [25]:
np.array(data[data.columns[0:-1]].values,
                              dtype=np.int16
                             )

array([[1, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 1, 1, 1]], dtype=int16)

In [24]:
data_ = torch.tensor(np.array(data[data.columns[0:-1]].values,
                              dtype=np.int16
                             ))

In [14]:
data_test = torch.tensor(data[data.columns[0:-1]].values).float()

In [33]:
data['prognosis'].unique()

array(['Fungal infection', 'Allergy', 'GERD', 'Chronic cholestasis',
       'Drug Reaction', 'Peptic ulcer diseae', 'AIDS', 'Diabetes ',
       'Gastroenteritis', 'Bronchial Asthma', 'Hypertension ', 'Migraine',
       'Cervical spondylosis', 'Paralysis (brain hemorrhage)', 'Jaundice',
       'Malaria', 'Chicken pox', 'Dengue', 'Typhoid', 'hepatitis A',
       'Hepatitis B', 'Hepatitis C', 'Hepatitis D', 'Hepatitis E',
       'Alcoholic hepatitis', 'Tuberculosis', 'Common Cold', 'Pneumonia',
       'Dimorphic hemmorhoids(piles)', 'Heart attack', 'Varicose veins',
       'Hypothyroidism', 'Hyperthyroidism', 'Hypoglycemia',
       'Osteoarthristis', 'Arthritis',
       '(vertigo) Paroymsal  Positional Vertigo', 'Acne',
       'Urinary tract infection', 'Psoriasis', 'Impetigo'], dtype=object)

In [39]:
Y = data['prognosis']

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,(vertigo) Paroymsal Positional Vertigo
4916,0,1,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,Acne
4917,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Urinary tract infection
4918,0,1,0,0,0,0,1,0,0,0,...,0,0,1,1,1,1,0,0,0,Psoriasis


In [41]:

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)


C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


array([15, 15, 15, ..., 38, 35, 27])

In [46]:
from keras.utils import np_utils
dummy_y = np_utils.to_categorical(encoded_Y)

In [56]:
data_[0].size()

torch.Size([132])

In [70]:
labels

tensor([15, 15, 15,  ..., 38, 35, 27])

In [50]:
from sklearn.model_selection import train_test_split

In [87]:
ANN = nn.Sequential(
    nn.Linear(132,264),
    nn.ReLU(),
    nn.Linear(264,264),
    nn.ReLU(),
    nn.Linear(264,41))

lossfun = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(ANN.parameters(),lr=.01)

In [90]:
numepochs = 1500

losses = torch.zeros(numepochs)
ongoingAcc = []

# loop over epochs
for epochi in range(numepochs):

  # forward pass
    yHat = ANN(data_test)

  # compute loss
    loss = lossfun(yHat,labels)
    losses[epochi] = loss

  # backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  # compute accuracy
    matches = torch.argmax(yHat,axis=1) == labels # booleans (false/true)
    matchesNumeric = matches.float()              # convert to numbers (0/1)
    accuracyPct = 100*torch.mean(matchesNumeric)  # average and x100 
    ongoingAcc.append( accuracyPct )              # add to list of accuracies



# final forward pass
predictions = ANN(data_test)
  
predlabels = torch.argmax(predictions,axis=1)
totalacc = 100*torch.mean((predlabels == labels).float())

In [91]:
totalacc

tensor(99.7561)

In [92]:
df_test = pd.read_csv("Testing.csv")

In [93]:
test = torch.tensor(df_test[df_test.columns[0:-1]].values).float()

In [94]:
Y_test = df_test['prognosis']
encoded_test_Y = encoder.transform(Y_test)

test_labels = torch.tensor(encoded_test_Y).long()

In [95]:
test_predictions = ANN(test)

In [97]:
test_predlabels = torch.argmax(test_predictions,axis=1)
test_totalacc = 100*torch.mean((test_predlabels == test_labels).float())

In [98]:
test_totalacc

tensor(100.)